In [1]:
import json
import pandas as pd
from datetime import datetime, timedelta

In [2]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.version import VERSION

def connect_to_snowflake():
  # Create Snowflake Session object
  print("Connecting to Snowflake...")
  connection_parameters = json.load(open('connection.json'))
  session = Session.builder.configs(connection_parameters).create()

  # Current Environment
  print("Current Environment...")
  snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
  snowpark_version = VERSION
  print('   User                        : {}'.format(snowflake_environment[0][0]))
  print('   Role                        : {}'.format(snowflake_environment[0][1]))
  print('   Database                    : {}'.format(snowflake_environment[0][2]))
  print('   Schema                      : {}'.format(snowflake_environment[0][3]))
  print('   Warehouse                   : {}'.format(snowflake_environment[0][5]))
  print('   Snowflake version           : {}'.format(snowflake_environment[0][4]))
  print('   Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))
  return session
session = connect_to_snowflake()


/Users/ayule/miniconda3/envs/snowflake/lib/python3.8/site-packages/snowflake/connector/options.py:107: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(


Connecting to Snowflake...
Current Environment...
   User                        : TAKEHOME_AY
   Role                        : TAKE_HOME_USER2
   Database                    : TAKE_HOME_USER2
   Schema                      : None
   Warehouse                   : TRANSFORM_WH
   Snowflake version           : 7.10.2
   Snowpark for Python version : 1.2.0


In [3]:
sensor_df = session.table("SENSOR.TIMESERIES")
readings_df = session.table("SENSOR.READINGS")
readings_pdf = readings_df.to_pandas()


In [4]:
readings_pdf.head()


,ALERT_STATUS,FILL_LEVEL,ID,DATA
0,"""HIGH""",113,"""049ab7f0-7a58-11e9-a7d5-79d7eb57b39a""","[\n {\n ""battery_level"": 3603,\n ""cente..."
1,"""LOW""",1,"""0581b640-80df-11e9-94cf-992c2615833b""","[\n {\n ""battery_level"": 3607,\n ""cente..."
2,"""LOW""",0,"""09147e60-9150-11e9-ac04-efcb622dc410""",[]
3,"""LOW""",0,"""0aeedbe0-80b6-11e9-8024-b1a347456020""","[\n {\n ""battery_level"": 3643,\n ""cente..."
4,"""LOW""",0,"""1044e8e0-9219-11e9-ae3b-42cf2a505c87""",[]


In [39]:
# should just use https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html
dummy_list = []
for index, row in readings_pdf[0:10].iterrows():
    readings = json.loads(row['DATA'])
    rolling_count = 0
    # rche_df.loc[index, 'wgs1984_latitude'] = dict_temp['lat']
    # print(readings)
    readings_df = pd.DataFrame.from_dict(readings)
    readings_df.rolling(window=4, min_periods=4)

    alerts = readings_df.rolling(window=4, min_periods=4)['fill_level_percentage']
    print(alerts)
    # services = readings_df.rolling(window=4, min_periods=4)['fill_level_percentage'].mean() < 20
    # print(alerts, services)
    # print(readings_df.head())
    # for reading in readings:
    #     print(reading)
    #     pass


0     False
1     False
2     False
3      True
4      True
      ...  
60    False
61    False
62    False
63    False
64    False
Name: fill_level_percentage, Length: 65, dtype: bool 0     False
1     False
2     False
3     False
4     False
      ...  
60    False
61    False
62    False
63    False
64    False
Name: fill_level_percentage, Length: 65, dtype: bool
0     False
1     False
2     False
3     False
4     False
      ...  
76    False
77    False
78    False
79    False
80    False
Name: fill_level_percentage, Length: 81, dtype: bool 0     False
1     False
2     False
3     False
4     False
      ...  
76    False
77    False
78    False
79    False
80    False
Name: fill_level_percentage, Length: 81, dtype: bool


KeyError: 'Column not found: fill_level_percentage'

In [7]:
readings_pdf.head()

,ALERT_STATUS,FILL_LEVEL,ID,DATA
0,"""HIGH""",113,"""049ab7f0-7a58-11e9-a7d5-79d7eb57b39a""","[\n {\n ""battery_level"": 3603,\n ""cente..."
1,"""LOW""",1,"""0581b640-80df-11e9-94cf-992c2615833b""","[\n {\n ""battery_level"": 3607,\n ""cente..."
2,"""LOW""",0,"""09147e60-9150-11e9-ac04-efcb622dc410""",[]
3,"""LOW""",0,"""0aeedbe0-80b6-11e9-8024-b1a347456020""","[\n {\n ""battery_level"": 3643,\n ""cente..."
4,"""LOW""",0,"""1044e8e0-9219-11e9-ae3b-42cf2a505c87""",[]


In [28]:
pd.json_normalize(sensor_pdf['V'])

""
0
1
2
3
4
...
6050
6051
6052
6053


In [5]:
# Load the JSON file into a Python dictionary

# Extract the relevant data
container_id = []
fill_level_percentage = []
timestamp = []

for reading in data['data']['data']:
    container_id.append(reading['container_id'])
    fill_level_percentage.append(reading['fill_level_percentage'])
    timestamp.append(datetime.strptime(reading['timestamp'], '%Y-%m-%d %H:%M:%S.%f'))

# Create a DataFrame from the extracted data
df = pd.DataFrame({
    'container_id': container_id,
    'fill_level_percentage': fill_level_percentage,
    'timestamp': timestamp
})

NameError: name 'data' is not defined

In [ ]:
# Identify when an alert is triggered and when a can is serviced
alerts = df.groupby('container_id').rolling(window=4, min_periods=4)['fill_level_percentage'].mean() > 80
services = df.groupby('container_id').rolling(window=4, min_periods=4)['fill_level_percentage'].mean() < 20

# Calculate the time taken for the garbage can to be serviced after an alert was sent to the waste management company
df['alert_triggered'] = alerts.groupby('container_id').transform(lambda x: x[x].index[0])
df['serviced'] = services.groupby('container_id').transform(lambda x: x[x].index[0])
df['pick_up_time'] = (df['serviced'] - df['alert_triggered']).dt.total_seconds() / 3600

# Determine if a can has overflowed
df['overflow'] = (df['fill_level_percentage'] > 100).rolling(window=3, min_periods=3).sum()
df['overflow'] = df.groupby('container_id')['overflow'].shift(-2)

# Filter out duplicate readings and missing time periods
df = df.drop_duplicates(subset=['container_id', 'timestamp']).set_index('timestamp')
df = df.groupby('container_id').resample('5min').ffill()

# Select the relevant columns and reset the index
df = df[['container_id', 'alert_triggered', 'serviced', 'pick_up_time', 'overflow']].reset_index()

# Convert the timestamps to the required format
df['alert_triggered'] = df['alert_triggered'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
df['serviced'] = df['serviced'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')

# Display the first few rows of the final dataset
print(df.head())